In [ ]:
%load_ext autoreload
%autoreload 2

import numpy as np
from omegaconf import OmegaConf
from src.models import models
from src.models import model as mainmodel
from src.models import modelConv2d
from src.models import modelineal
from src.models import decoders
from src import loss_func
from src import train
from src import loader
import matplotlib.pyplot as plt
import torch
import torch.nn as nn
from src import optimizer_Factory as of
import Data.genData as genData
from src import custom_plots as cp
from src import Visual_utils as vu
import torchvision
import wandb
import random
import os

from scipy.signal import savgol_filter


os.environ["PYTORCH_CUDA_ALLOC_CONF"] = "max_split_size_mb:2000"
torch.cuda.empty_cache() 
torch.manual_seed(42)

## Create Dataset

In [ ]:
import cv2

# load video from folder using opencv
video = cv2.VideoCapture('Data/15cm65g30deg.mov')
frames = []
while True:
    ret, frame = video.read()
    if ret == False:
        break
    frame = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
    frames.append(frame)
video.release()
print(len(frames))

In [ ]:
dataset = []
np_frames = np.array(frames)

nframes_per_sample = 30
total_samples = 5000
for i in range(total_samples):

    frames_temp = np_frames[i:i+nframes_per_sample]
    
    dataset.append(np.expand_dims( frames_temp, axis=1))

dataset = np.array(dataset)
print(dataset.shape)

In [ ]:
np.save('Data/name.npy', dataset)

## Train Model

In [ ]:
torch.cuda.empty_cache() 
torch.manual_seed(0)

data_folder = np.load('Data/name.npy')
data_train = data_folder
print(data_train.shape)
dt = 3/30
train_dataloader, test_dataloader, train_x, val_x  = loader.getLoader_folder(data_train, split=True)




latentEncoder_I = mainmodel.EndPhys(dt = dt,  
                                pmodel = "Damped_oscillation",
                                init_phys = 1.0, 
                                initw=True)

latentEncoder_I, log, params  = train.train(latentEncoder_I, 
                                train_dataloader, 
                                test_dataloader,
                                init_phys = 1.0,                                 
                                loss_name='latent_loss')

## visualize angle

In [ ]:
dataloader  = loader.getLoader_folder(data_train, split=False)

z = None

device = "cuda" if torch.cuda.is_available() else "cpu"

model = mainmodel.EndPhys(dt = dt,  
                                pmodel = "Damped_oscillation",
                                init_phys = 1.0, 
                                initw=True)

model.load_state_dict(latentEncoder_I.state_dict())
model.to(device)

z = None
X = []

for data in dataloader:

    input_Data, out_Data = data

    x0 = input_Data

    x0 = x0.to(device=device, dtype=torch.float)

    x2 = out_Data.to(device=device, dtype=torch.float)

    outputs = model(x0)
    z2_encoder, z2_phys, z3=outputs

    if z is None:
        z = z2_encoder.detach().cpu().numpy()[0][0]
    else:
        z = np.vstack((z,z2_encoder.detach().cpu().numpy()[0][0]))

In [ ]:
plt.figure(figsize=(20,5))
time = np.arange(z.shape[0])
plt.scatter(time*dt, z, label='real',s=4)
plt.plot(time*dt, z, label='real')
plt.xlabel('Time')
plt.ylabel('Latent Space (angle)')
plt.show()